In [1]:
import pandas as pd
import numpy as np

In [2]:
lpz = pd.read_csv('vendas_modificado.csv')

In [3]:
lpz.head()

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"R$ 16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"R$ 3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"R$ 1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,"R$ 8,74",4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,"R$ 9,48",2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


In [4]:
print(lpz.dtypes)

id_da_compra      int64
data             object
hora             object
cliente          object
produto          object
valor            object
quantidade        int64
total           float64
status           object
cidade           object
estado           object
pais             object
cep              object
frete           float64
pagamento        object
vendedor         object
marca            object
dtype: object


## <p align="center"><strong><span style="color:green;"> Verificação e Padronização de Tipos e Formatos</span></strong></p>

#### <p align="center"><strong>FORMATAÇÃO DE DATA E HORA</strong></p>

In [5]:
lpz['data'] = pd.to_datetime(lpz['data'], errors='coerce').dt.strftime('%Y-%m-%d')

In [6]:
lpz['hora'] = pd.to_datetime(lpz['hora'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')

In [7]:
print(lpz[['data', 'hora']].head())

         data      hora
0  2021-03-20  23:35:51
1  2020-10-30  09:00:53
2  2021-06-09  15:30:28
3  2022-06-04  08:41:23
4  2019-05-04  13:38:45


#### <p align="center"><strong>VALORES NUMÉRICOS</strong></p>

In [8]:
# Convertendo colunas para formato numérico
colunas_numericas = ['quantidade', 'total', 'frete']

for coluna in colunas_numericas:
    lpz[coluna] = pd.to_numeric(lpz[coluna], errors='coerce')

# Verificando os tipos de dados após a conversão
print(lpz.dtypes)

id_da_compra      int64
data             object
hora             object
cliente          object
produto          object
valor            object
quantidade        int64
total           float64
status           object
cidade           object
estado           object
pais             object
cep              object
frete           float64
pagamento        object
vendedor         object
marca            object
dtype: object


## <p align="center"><strong><span style="color:purple;">Tratamento de Inconsistências</span></strong></p>

!! Antes de iniciar o tratamento de valores ausentes irei corrijir a coluna produto pois pode ajudar com os valores ausentes !!

In [9]:
print(lpz[['produto']].head())

            produto
0  Queijo Mussarela
1   Molho de Tomate
2      Água Mineral
3            Carvão
4              Café


Importação da biblioteca (re) para trabalhar com expressões regulares 

In [10]:
import re

In [11]:
# Remover caracteres especiais da coluna 'produto'
lpz['produto'] = lpz['produto'].astype(str).apply(
    lambda x: re.sub(r'[^A-Za-z0-9À-ÿ\s]', '', x)
)

In [12]:
from fuzzywuzzy import process

produtos_unicos = lpz['produto'].unique()
for prod in produtos_unicos:
    print(process.extract(prod, produtos_unicos, limit=3))

[('Queijo Mussarela', 100), ('Qbeijo Mussarela', 94), ('Queijo Mussarelz', 94)]
[('Molho de Tomate', 100), ('Mopho de Tomate', 93), ('Molmo de Tomate', 93)]
[('Água Mineral', 100), ('Água Mineras', 91), ('Água Mineual', 91)]
[('Carvão', 100), ('Açúcar', 67), ('Macarrão', 67)]
[('Café', 100), ('Caft', 86), ('Caff', 86)]
[('Feijão', 100), ('ieijão', 80), ('Queijo Mussarela', 72)]
[('Condicionador', 100), ('Condicioiador', 92), ('Condibionador', 92)]
[('Sabonete', 100), ('zabonete', 88), ('Sabonepe', 88)]
[('Açúcar', 100), ('Macarrão', 90), ('Açúcaz', 75)]
[('Manteiga', 100), ('Manteigt', 88), ('Amaciante', 59)]
[('Papel Higiênico', 100), ('Papel Toalha', 56), ('Papel qoalha', 56)]
[('Óleo de Soja', 100), ('Pão de Forma', 64), ('Molho de Tomate', 62)]
[('Vinho', 100), ('Papel Higiênico', 54), ('Farinha de Trigo', 54)]
[('Macarrão', 100), ('Açúcar', 90), ('Macawrão', 86)]
[('Shampoo', 100), ('Sal', 60), ('Sabão em Pó', 50)]
[('Presunto', 100), ('Presuntd', 88), ('Papel Toalha', 45)]
[('Cer

In [13]:
correcao_produto = {
    "Qbeijo Mussarela": "Queijo Mussarela",
    "Queijo Mussarelz": "Queijo Mussarela",
    "Mopho de Tomate": "Molho de Tomate",
    "Água Mineras": "Água Mineral",
    "Água Mineual": "Água Mineral",
    "Condicioiador": "Condicionador",
    "Condibionador": "Condicionador",
    "zabonete": "Sabonete",
    "Sabonepe": "Sabonete",
    "Macawrão": "Macarrão",
    "Manteigt": "Manteiga",
    "Papel qoalha": "Papel Toalha",
    "Papel Twalha": "Papel Toalha",
    "Presuntd": "Presunto",
    "Amaciayte": "Amaciante",
    "Desinfetanue": "Desinfetante",
    "Desinfekante": "Desinfetante",
    "Refrigkrante": "Refrigerante",
    "Arroc": "Arroz",
    "Deqergente": "Detergente",
    "Deterwente": "Detergente",
    "Cafc": "Café",
    "Caft": "Café"
}
lpz['produto'] = lpz['produto'].replace(correcao_produto)

In [14]:
print(lpz[['produto']].head())

            produto
0  Queijo Mussarela
1   Molho de Tomate
2      Água Mineral
3            Carvão
4              Café


#### <p align="center"><strong>VALORES AUSENTES</strong></p>

In [15]:
print(lpz.isnull().sum())

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3685
status             0
cidade             0
estado             0
pais               0
cep                0
frete           7371
pagamento          0
vendedor        3680
marca              0
dtype: int64


VALORES FALTANTES E ESCOLHA DO TRATAMENTO DEPENDENDO DA COLUNA<br>
TORTAL: RECALCULAR SE VALOR,QUANTIDADE E FRETE SE ESTIVEREM CORRETOS <br>
FRETE: MEDIA <br>
VENDEDOR: REMOÇÃO OU APLICAR A PALAVRA DESCONHECIDO <br>

##### <p align="center"><strong>TRATAMENTO DE VALOR</strong></p>

In [16]:
lpz['valor'] = lpz['valor'].str.replace(r'R\$', '', regex=True).str.strip()

In [17]:
lpz.head(3)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba


##### <p align="center"><strong>CORREÇÃO DOS ESTADOS</strong></p>

In [18]:
estado_correto = {
    'São Paulo': 'SP',
    'MTSa': 'MT',
    'PSC': 'SC'
}
lpz['estado'] = lpz['estado'].replace(estado_correto)

##### <p align="center"><strong>TRATAMENTO DO FRETE</strong></p>

In [19]:
lpz['Frete_Check1'] = lpz.groupby(['cidade', 'estado'])['frete'].transform('median')

In [20]:
lpz.head(3)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,Frete_Check1
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre,0.0
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini,0.0
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba,22.0


In [21]:
lpz['Frete_Check2'] = lpz.frete.fillna(lpz.groupby (['cidade', 'estado'])['frete'].transform('median'))

In [22]:
lpz = lpz[(lpz['frete'] >= 0) & (lpz['total'] >= 0)]

In [23]:
lpz.head(3)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,Frete_Check1,Frete_Check2
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre,0.0,20.0
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini,0.0,0.0
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba,22.0,22.0


In [24]:
lpz['frete'] =   lpz.frete.fillna(lpz.groupby (['cidade', 'estado'])['frete'].transform('median'))

In [25]:
lpz = lpz.drop(columns=['Frete_Check1', 'Frete_Check2'])

In [26]:
lpz.head(3)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba


In [27]:
print(lpz.isnull().sum())

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total              0
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor        3518
marca              0
dtype: int64


##### <p align="center"><strong>VERIFICAÇÃO DO VALOR TOTAL</strong></p>

In [28]:
def corrigir_valor(valor):
    match = re.search(r'\d{1,3}(,\d{2})?', str(valor))  # pega o primeiro número com vírgula decimal
    if match:
        return match.group().replace(',', '.')
    return np.nan

lpz['valor'] = lpz['valor'].apply(corrigir_valor).astype(float)

In [29]:
# Calcular o total correto
lpz['total_calculado'] = lpz['valor'] * lpz['quantidade'] + lpz['frete']

In [30]:
lpz[lpz.total.isnull()].head(6)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,total_calculado


In [31]:
lpz['total'] = np.where(
    np.abs(lpz['total'] - lpz['total_calculado']) < 0.01,
    lpz['total'],
    lpz['total_calculado']
)

In [32]:
lpz.drop(columns=['total_calculado'], inplace=True)

In [33]:
lpz.head(6)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,Café,10.16,1,20.16,Entregue com Sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações


In [34]:
print(lpz.isnull().sum())

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total              0
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor        3518
marca              0
dtype: int64


##### <p align="center"><strong>TRATAMENTO DO VENDEDOR</strong></p>

In [35]:
lpz[lpz.vendedor.isnull()].head(6)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
112,52388,2018-07-30,07:10:51,MIGUEL ANTÔNIO DE DECCO OLIVEIRA,Manteiga,8.77,5,56.85,Pagamento Confirmado,Juiz de Fora,MG,Brasil,36000-000,13.0,Cartão de Crédito,NaN,Aviação
425,46034,2019-12-28,14:14:40,RAFAEL HENRIQUE COSTA BARNABE MARAZO,Suco de Laranja,825.00,5,4125.00,Aguardando Pagamento,Leopoldina,MG,Brasil,367XXXXX,0.0,Boleto,NaN,Prats
525,30444,2022-09-11,21:39:42,LUCAS GONÇALVES DONADONI,Café,10.33,2,20.66,Em Separação,Niterói,RJ,Brasil,24000-000,-0.0,Transferência Bancária,NaN,3 Corações
543,41142,2020-10-16,12:47:08,GABRIEL CARLOS DE SOUZA,Vinho,28.89,1,28.89,Pagamento Confirmado,Coronel Xavier Chaves,MG,Brasil,36330-000,0.0,Cartão de Crédito,NaN,Miolo
544,194,2020-06-28,12:28:55,MATEUS Araujo Kuhn,Suco de Laranja,6.86,6,51.16,Em Separação,Lima Duarte,MG,Brasil,36140-000,10.0,Pix,NaN,Prats
623,55288,2020-05-08,14:08:29,SAMUEL HENRIQUE CAÇADOR,Queijo Mussarela,17.69,1,17.69,Entregue,Niterói,RJ,Brasil,24000-000,0.0,Transferência Bancária,NaN,Quatá


In [36]:
lpz['vendedor'] = lpz['vendedor'].fillna('DESCONHECIDO')

In [37]:
lpz.head(6)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,Café,10.16,1,20.16,Entregue com Sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações


##### <p align="center"><strong>TRATAMENTO DE DUPLICATAS</strong></p>

In [38]:
# Remover duplicatas com base em todas as colunas
lpz = lpz.drop_duplicates()

##### <p align="center"><strong>STATUS INVÁLIDOS</strong></p>

In [39]:
status_correto = {
    'PC': 'Pagamento Confirmado',
    'Pgto Confirmado': 'Pagamento Confirmado',
    'AP': 'Aguardando Pagamento',
    'Aguardando Pgto': 'Aguardando Pagamento',
    'aguardando pagamento': 'Aguardando Pagamento',
    'Sep': 'Em Separação',
    'Separando': 'Em Separação',
    'Transp': 'Em Transporte',
    'Transportando': 'Em Transporte',
    'Entg': 'Entregue com Sucesso',
    'Entregue': 'Entregue com Sucesso'
}
lpz['status'] = lpz['status'].replace(status_correto)

##### <p align="center"><strong>TRATAMENTO DE CEPs</strong></p>

In [40]:
lpz = lpz[lpz['cep'].astype(str).str.match(r'^\d{5}-\d{3}$')]

##### <p align="center"><strong>REMOÇÃO DE ESPAÇOS EXTRAS EM TODAS AS COLUNAS</strong></p>

In [41]:
for col in lpz.select_dtypes(include='object').columns:
    lpz[col] = lpz[col].str.strip()

In [42]:
lpz.to_csv('final_corrigido.csv', index=False, encoding='utf-8-sig')


## <p align="center"><strong><span style="color:red;">Relatório sobre a limpeza!!</span></strong></p>

🔧 Etapas da Limpeza Realizadas
1. Padronização de Tipos e Formatos

    Datas:

        Todas as datas foram convertidas para o formato YYYY-MM-DD.

    Horas:

        Horários convertidos para o padrão HH:MM:SS.

    Valores Numéricos:

        As colunas valor, quantidade, frete e total foram convertidas para tipo numérico (float ou int).

        Valores com símbolos (ex: "R$", ",") foram limpos para manter apenas os números com . como separador decimal.

        Importação da biblioteca (re) para facilitar na limpeza de caracteres nas colunas.

2. Tratamento de Inconsistências

    Valores Ausentes:

        Registros com campos essenciais ausentes (ex: valor, quantidade) foram removidos.

        Dados faltantes não essenciais foram preenchidos com estratégias específicas (ex: frete com média, datas com modo).

    Registros Duplicados:

        Foram identificados e removidos X registros duplicados, mantendo apenas a versão mais recente ou completa.

    Verificação de Cálculo do Total:

        Aplicada a fórmula:
        total = valor * quantidade + frete

        Registros com cálculos incorretos foram corrigidos

    Tratamento do nome do Vendedor:

        Conforme vimos, há uma inconsistência em relação ao nome dos vendedores. Nesse caso, colocamos eles como DESCONHECIDO ao realizar a limpeza..

    Tratamento do nome dos Produtos:

        Padronização do nomes dos produtos para uma analise mais facil

    Tratamento de Estados e Status:

        Correção de estados e status incorretos

    Remoção de espaços:

        Remoção de espaços extras em todas as colunas
